In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, expr
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, IntegerType, ArrayType, DateType
import sys
import os
from pyspark.sql import DataFrame
from pyspark.sql.utils import AnalysisException
from delta.tables import *
import io
import json

In [0]:
def create_spark_session():
    return  SparkSession.builder\
    .config('spark.executor.memory', '8G')\
.   getOrCreate()

In [0]:
spark = create_spark_session()

O catálogo de metadados do Spark pode ser acessado pelo objeto

`SparkSession.catalog` 

As principais funcionalidades são:

* `listDatabases()`: lista todas os databases disponíveis;
* `listTables()`: lista todas as tabelas disponíveis em um determinado database;
* `listFucntions()`: lista as funções disponíveis em um determinado database;
* `refreshTable()`: atualiza os metadados de uma determinada tabela
* `uncacheTable()`: remove uma tabela salva em memória
* `clearCache()`: remove todas as tabelas salvas em memória

In [0]:
spark.catalog.listDatabases()

Out[4]: [Database(name='default', catalog='spark_catalog', description='Default Hive database', locationUri='dbfs:/user/hive/warehouse')]

In [0]:
spark.catalog.listTables('default')

Out[5]: []

### Show e Create Databases

Os databases do Spark são uma ferramenta para organizar tabelas. Eles podem e devem ser vistos como algo muito próximo dos databases de servidores de bancos de dados relacionais. O Spark utiliza por padrão um database chamado default, que serve para criar tabelas, views e realizar consultas caso o usuário não tenha definido o seu próprio. Um ponto importante é que essas estruturas persistem em diferentes sessões: se o usuário mudar de database, todas as tabelas permanecerão no database anterior e vão precisar ser consultadas de maneira diferente.

Existem alguns comandos do SQL importantes na hora de se trabalhar com databases. Else são:

* `SHOW DATABASES`: lista todas os databases disponíveis, de forma análoga ao Catalog ;
* `CREATE DATABASE <nome_do_db>`: cria um database
* `USE <nome_do_db>`: define o database como o atual para a realização de queries
    * **Obs**: ao se mudar de database, é possível acessar tabelas de um database anterior usando o prefixo “nome_do_db.” antes do nome da tabela. Exemplo:
        ```
        USE db2
        SELECT * FROM db1.table
        ```
* `SELECT current_database()`: retorna qual o database definido como o atual
* `DROP DATABASE IF EXISTS <nome_do_db>`: deleta determinado database dentre aqueles que foram definidos. Atenção: nunca delete o database default do Spark.


In [0]:
spark.sql("""
DROP DATABASE IF EXISTS department;
""")

Out[35]: DataFrame[]

In [0]:
spark.sql("""
CREATE DATABASE IF NOT EXISTS department;
""")

Out[37]: DataFrame[]

In [0]:
spark.catalog.listDatabases()

Out[40]: [Database(name='default', catalog='spark_catalog', description='Default Hive database', locationUri='dbfs:/user/hive/warehouse'),
 Database(name='department', catalog='spark_catalog', description='', locationUri='dbfs:/user/hive/warehouse/department.db')]

In [0]:
spark.catalog.listTables('department')

Out[38]: []

In [0]:
path_countries = '/FileStore/transient/departments/countries'
df_countries = spark.read.format('csv')\
.option("header", True)\
.option("sep", ",")\
.option("quote","\'")\
.option("inferSchema",True)\
.load(path_countries)

In [0]:
path_regions = '/FileStore/transient/departments/regions'
df_regions = spark.read.format('csv')\
.option("header", True)\
.option("sep", ",")\
.option("quote","\'")\
.option("inferSchema",True)\
.load(path_regions)

### Tables

* **Managed Tables**: o Spark administra tanto os dados quanto os metadados das tabelas, de forma que operações como DROP TABLE afetam também os dados escritos em disco;
* **Unmanaged Tables**: o Spark administra somente os metadados da tabela, e os dados escritos em disco não são alterados em nenhum momento

In [0]:
spark.sql("""
USE department;
""")

Out[11]: DataFrame[]

In [0]:
spark.catalog.listTables('department')

Out[14]: [Table(name='regions', catalog='spark_catalog', namespace=['department'], description=None, tableType='EXTERNAL', isTemporary=False)]

**Criando Unmanaged Tables** </br>
Para criar uma tabela unmanaged (Não gerenciada) , passe o caminho do storage durante o comando write, e adicione saveAsTable('NOME_TABELA')

In [0]:
df_regions.write.option('path', 'dbfs:/FileStore/bronze/sql_db/departments/regions').format('parquet').mode('overwrite').saveAsTable("regions")

In [0]:
df_regions.printSchema()

root
 |-- region_id: integer (nullable = true)
 |-- region_name: string (nullable = true)



In [0]:
spark.sql("""    
    create table if not exists regions_2(
        region_id   INT,
        region_name STRING
    )using parquet options (PATH "dbfs:/FileStore/bronze/sql_db/departments/regions")
""")

Out[57]: DataFrame[]

In [0]:
spark.catalog.listTables('department')

Out[56]: [Table(name='regions', catalog='spark_catalog', namespace=['department'], description=None, tableType='EXTERNAL', isTemporary=False)]

In [0]:
spark.sql("""
select * from regions_2 limit 5;
""").show()   

+---------+--------------------+
|region_id|         region_name|
+---------+--------------------+
|        1|              Europe|
|        2|            Americas|
|        3|                Asia|
|        4|Middle East and A...|
+---------+--------------------+



In [0]:
spark.sql("""
DROP TABLE if EXISTS regions_2;
""")

spark.sql("""
DROP TABLE if EXISTS regions;
""")

Out[64]: DataFrame[]

In [0]:
spark.catalog.listTables('department')

Out[65]: []

In [0]:
dbutils.fs.ls('dbfs:/FileStore/bronze/sql_db/departments/regions')


Out[66]: [FileInfo(path='dbfs:/FileStore/bronze/sql_db/departments/regions/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1707821107000),
 FileInfo(path='dbfs:/FileStore/bronze/sql_db/departments/regions/_committed_3359770175017864369', name='_committed_3359770175017864369', size=123, modificationTime=1707821107000),
 FileInfo(path='dbfs:/FileStore/bronze/sql_db/departments/regions/_started_3359770175017864369', name='_started_3359770175017864369', size=0, modificationTime=1707821107000),
 FileInfo(path='dbfs:/FileStore/bronze/sql_db/departments/regions/part-00000-tid-3359770175017864369-29547bcf-6fee-481e-babc-ec07e74eab75-21-1-c000.snappy.parquet', name='part-00000-tid-3359770175017864369-29547bcf-6fee-481e-babc-ec07e74eab75-21-1-c000.snappy.parquet', size=945, modificationTime=1707821107000)]

In [0]:
spark.sql("""
select * from regions limit 5;
""").display()

+---------+--------------------+
|region_id|         region_name|
+---------+--------------------+
|        1|              Europe|
|        2|            Americas|
|        3|                Asia|
|        4|Middle East and A...|
+---------+--------------------+



**Criando Managed Tables**</br>
Para criar uma tabela managed (gerenciada) , basta omitir o caminho durante o comando saveAsTable('NOME_TABELA')

In [0]:
df_regions.write.format('parquet').mode('overwrite').saveAsTable("regions")

In [0]:
spark.sql("""    
    create table if not exists regions(
        region_id   INT,
        region_name STRING
    )using parquet;
""")

Out[83]: DataFrame[]

In [0]:
spark.sql(""" 
            insert into department.regions values
            (1,'Never Land');
        """)

Out[86]: DataFrame[]

In [0]:
spark.sql("""
select * from regions limit 5;
""").display()

region_id,region_name
1,Never Land


**Detalhe sobre Managed Tables**</br>
Quando utilizamos uma Managed Tables, os dados precisam ser armazenados em algum lugar do cluster, no databricks o padrão é : </br>
**'/user/hive/warehouse/meu_database.db/minha_tabela'** </br>
Esse local pode ser modificado informado a **config('spark.sql.warehouse.dir', 'caminho_warehouse')** 

In [0]:

dbutils.fs.ls('/user/hive/warehouse/department.db/regions')

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-1220161194661673>:1
----> 1 dbutils.fs.ls('dbfs://user/hive/warehouse/department.db/regions')

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = None
--> 362 raise exc

ExecutionError: An error occurred while calling o812.ls.
: java.io.FileNotFoundException: /user/hive/warehouse/department.db/regions
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:121)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:69)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV1.listStatus(DatabricksFileSystemV1.scala:179)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.listStatus(D

In [0]:
spark.sql("""
DROP TABLE regions;
""")


Out[88]: DataFrame[]

**Criando Views**

In [0]:
df_countries.createOrReplaceTempView('countries_view')

In [0]:
spark.sql("""
select * from countries_view;
""").display()


country_id,country_name,region_id
AR,Argentina,2
AU,Australia,3
BE,Belgium,1
BR,Brazil,2
CA,Canada,2
CH,Switzerland,1
CN,China,3
DE,Germany,1
DK,Denmark,1
EG,Egypt,4


In [0]:
df_countries.createOrReplaceGlobalTempView('countries_global_view')

**Utilizando a interface SQL**

In [0]:
spark.sql("""
  DROP TABLE IF EXISTS countries ;
""")

Out[93]: DataFrame[]

In [0]:
spark.sql("""
  CREATE TABLE countries (
  country_id STRING, 
  country_name STRING,
  region_id INTEGER
)using PARQUET ;
""")

Out[94]: DataFrame[]

In [0]:
spark.catalog.listTables('department')

Out[95]: [Table(name='countries', catalog='spark_catalog', namespace=['department'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='countries_view', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

**Inserindo registros na tabela** </br>

In [0]:
spark.sql(""" 
            insert into department.countries values
            ('100','Never Land',99)
        """)

Out[96]: DataFrame[]

In [0]:
spark.sql("SELECT * FROM department.countries limit 5").display()

country_id,country_name,region_id
100,Never Land,99


In [0]:
spark.sql(""" 
        insert into department.countries
        select        
              country_id , 
              country_name ,       
              region_id      
        from countries_view
        """)

Out[100]: DataFrame[]

In [0]:
spark.sql("SELECT * FROM department.countries limit 50").display()

country_id,country_name,region_id
AR,Argentina,2
AU,Australia,3
BE,Belgium,1
BR,Brazil,2
CA,Canada,2
CH,Switzerland,1
CN,China,3
DE,Germany,1
DK,Denmark,1
EG,Egypt,4


In [0]:
spark.sql(""" 
        insert into department.countries values
            ('ZW','Zimbabwe',2)   
        """)

Out[104]: DataFrame[]

**Consultas, joins, agregações**</br>
Alem de criar databases, apagar databases, criar views e inserir valores em tabelas, todos os comandos referentes a manipulação de dados e consultas do SQL padrão </br>
esta disponível no spark sql, possibilitando criar agregações, joins, unions, filtros , alem de utilizar todas as funções padrões o sql para manipulação de string, numeros, etc.</br>

Cada consulta realizada no spark sql retorna um dataframe spark como resultado, sendo possivel salvar o resultado de uma consulta no datalake o apenas exibir o resultado.</br>
É importante ter em mente que o spark sql padrão ** NÃO PERMITE ** operações de update , merge , delete , remover ou atualizar valores é necessário sempre sobrescrever o 
resultado previamente escrito.



In [0]:
spark.sql(""" 
        delete from department.countries where country_id = 'ZW' ;
        """)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1220161194661688>:1
----> 1 spark.sql(""" 
      2         delete from department.countries where country_id = 'ZW' ;
      3         """)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/session.py:1387, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1385 try:
   1386     litArgs = {k: _to_java_column(lit(v)) for k, v in (args or {}).items()}
-> 1387     return DataFrame(self._jsparkSession.sql(sqlQuery, litArgs), self)
   1388 finally:
   1389     if le

In [0]:
spark.sql(""" 
        update department.countries set region_id = 2 where country_id = 'ZW' ;
        """)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1220161194661689>:1
----> 1 spark.sql(""" 
      2         update department.countries set region_id = 2 where country_id = 'ZW' ;
      3         """)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/session.py:1387, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1385 try:
   1386     litArgs = {k: _to_java_column(lit(v)) for k, v in (args or {}).items()}
-> 1387     return DataFrame(self._jsparkSession.sql(sqlQuery, litArgs), self)
   1388 finally:
   1

In [0]:
df_regions.write.option('path', 'dbfs:/FileStore/bronze/sql_db/departments/regions').format('parquet').mode('overwrite').saveAsTable("department.regions")

In [0]:
spark.sql("SELECT * FROM department.regions limit 50").display()

region_id,region_name
1,Europe
2,Americas
3,Asia
4,Middle East and Africa


In [0]:
spark.sql("SELECT * FROM department.countries limit 50").display()

country_id,country_name,region_id
AR,Argentina,2
AU,Australia,3
BE,Belgium,1
BR,Brazil,2
CA,Canada,2
CH,Switzerland,1
CN,China,3
DE,Germany,1
DK,Denmark,1
EG,Egypt,4


In [0]:
spark.sql('SHOW TABLES;').display()

database,tableName,isTemporary
department,countries,false
department,regions,false
,countries_view,true


In [0]:
spark.sql("""         
        select        
              *
        from countries a
        inner join regions b on
        a.region_id = b.region_id
        """).display()

country_id,country_name,region_id,region_id,region_name
AR,Argentina,2,2,Americas
AU,Australia,3,3,Asia
BE,Belgium,1,1,Europe
BR,Brazil,2,2,Americas
CA,Canada,2,2,Americas
CH,Switzerland,1,1,Europe
CN,China,3,3,Asia
DE,Germany,1,1,Europe
DK,Denmark,1,1,Europe
EG,Egypt,4,4,Middle East and Africa


In [0]:
spark.sql("""         
        select        
              count(*) as qtd_paises,
              region_name
        from countries a
        inner join regions b on
        a.region_id = b.region_id
        group by
        region_name
        """).display()

qtd_paises,region_name
8,Europe
6,Middle East and Africa
6,Americas
6,Asia
